In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd

In [2]:
%load_ext google.cloud.bigquery

In [3]:
key_path = "big-data-analytics-b0511a030e6d.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

bqclient = bigquery.Client(
    credentials=credentials,
    project=credentials.project_id,
)

In [4]:
company='twitter'

In [7]:
query_post = (
    "SELECT id, title, selftext, num_comments FROM `fh-bigquery.reddit_posts.2019_03` "
    "WHERE subreddit='news' AND LOWER(title) LIKE LOWER('%" + company + "%')"
)

job_post = bqclient.query(
    query_post,
    location="US",
)  # API request - starts the query

df_post = (
    job_post
    .result()
    .to_dataframe()
)

print(df_post.head())

       id                                              title selftext  \
0  b4i80h  Wie kann man die Blockierung auf Twitter aufhe...            
1  b4ia9n  SPYING! Facebook Google and Twitter Owned Oper...            
2  b256fo  5 Types of Stories Brands &amp; Marketers Shou...            
3  azo5wi  Internet Collapses in Venezuela with 80% Offli...            
4  azu76s                  TWITTER CLIPS from Islamic Europe            

   num_comments  
0             0  
1             0  
2             0  
3             0  
4             0  


In [9]:
comments = []
for i in df_post[df_post.num_comments > 0].id:
    query_comments = (
        "SELECT body, parent_id, ups, downs, score FROM `fh-bigquery.reddit_comments.2019_03` "
        "WHERE subreddit='news' AND parent_id LIKE '%" + i + "'"
    )

    job_comments = bqclient.query(
        query_comments,
        location="US",
    )  # API request - starts the query

    df_comments = (
        job_comments
        .result()
        .to_dataframe()
    )
    
    comments.append(df_comments)

df_comments = pd.concat(comments, ignore_index=True)

In [11]:
print(df_comments.head())

                                                body  parent_id   ups downs  \
0  [I think we have bigger anti vaxxer idiots to ...  t3_b0klc7  None  None   
1  Unsure how an anti-vaxer could possibly assist...  t3_b0klc7  None  None   
2  Fuck this article and fuck people who just bli...  t3_b0klc7  None  None   
3                          Take away his blue check!  t3_b0klc7  None  None   
4    Is twitter going to start blocking pro vaxxers?  t3_b0klc7  None  None   

   score  
0     10  
1      3  
2      4  
3      1  
4      2  
